In [9]:
!pip install fiftyone ultralytics

In [10]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

In [11]:
import numpy as np
import os
from tqdm import tqdm

In [12]:
from ultralytics import YOLO

detection_model = YOLO("yolov8n.pt")
seg_model = YOLO("yolov8n-seg.pt")

In [13]:
results = detection_model("https://ultralytics.com/images/bus.jpg")


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 d:\Genta\ML_practice\voxel\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 28.7ms
Speed: 3.2ms preprocess, 28.7ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 480)


In [30]:
import fiftyone as fo
import kagglehub
import os
from pathlib import Path
from ultralytics import YOLO

# Download Kaggle dataset
# Download latest version
path = kagglehub.dataset_download("farzadnekouei/pothole-image-segmentation-dataset")

print("Path to dataset files:", path)

# Create a new FiftyOne dataset
dataset = fo.Dataset("potholes_detection")

# Find image files in the downloaded dataset
image_dir = Path(path)
image_paths = []
for ext in ['.jpg', '.jpeg', '.png']:
    image_paths.extend(list(image_dir.glob(f"**/*{ext}")))

# Add images to the dataset
for image_path in image_paths:
    sample = fo.Sample(filepath=str(image_path))
    dataset.add_sample(sample)

print(f"Added {len(dataset)} images to the dataset")

# Load a YOLOv8 model
model = YOLO("yolov8l.pt")

# Apply the model to the dataset
dataset.apply_model(model, label_field="yolov8l")

# Launch the App to visualize the results
session = fo.launch_app(dataset)

dataset_name = dataset.name

100%|██████████| 59.3M/59.3M [01:13<00:00, 840kB/s] 

Extracting files...


Path to dataset files: C:\Users\altha\.cache\kagglehub\datasets\farzadnekouei\pothole-image-segmentation-dataset\versions\2


ValueError: Dataset name 'potholes_detection' is not available

In [24]:
fo.delete_dataset(dataset_name)